In [1]:
%pwd

'D:\\Dropbox\\Ted 2017\\11A_技術\\[TREND] Data Science and ML\\[Tutorials] Kaggle\\[Competitions] Kaggle Competitions\\2016-10_ImgClass_The Nature Fisheries Monitoring\\code'

In [2]:
#Create references to important directories we will use over and over
import os, sys
current_dir = os.getcwd()
CODE_HOME_DIR = current_dir
DATA_HOME_DIR = CODE_HOME_DIR + '/../input/'
print(CODE_HOME_DIR)
print(DATA_HOME_DIR)

D:\Dropbox\Ted 2017\11A_技術\[TREND] Data Science and ML\[Tutorials] Kaggle\[Competitions] Kaggle Competitions\2016-10_ImgClass_The Nature Fisheries Monitoring\code
D:\Dropbox\Ted 2017\11A_技術\[TREND] Data Science and ML\[Tutorials] Kaggle\[Competitions] Kaggle Competitions\2016-10_ImgClass_The Nature Fisheries Monitoring\code/../input/


In [3]:
#Allow relative imports to directories above this folder/
sys.path.insert(1,os.path.join(sys.path[0], '..'))

#import modules
import utils
from utils import *
# from vgg16_ted import vgg16BN
%matplotlib inline

Using TensorFlow backend.


#### Setup Paths

In [5]:
%cd $DATA_HOME_DIR

#Set path to sample/ path if desired
path = DATA_HOME_DIR  + '/sample/' # + '/' 
result_path = path + 'results/'
train_path = path + 'train/'
valid_path = path + 'valid/'
test_path = path + 'test/'
model_path = path + 'models/'

D:\Dropbox\Ted 2017\11A_技術\[TREND] Data Science and ML\[Tutorials] Kaggle\[Competitions] Kaggle Competitions\2016-10_ImgClass_The Nature Fisheries Monitoring\input


In [6]:
batch_size = 8 # batch_size = 64

In [7]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

(trn_classes, val_classes, trn_labels, val_labels, 
    filenames, val_filenames, test_filenames) = get_classes(path)

Found 400 images belonging to 8 classes.
Found 200 images belonging to 8 classes.
Found 400 images belonging to 8 classes.
Found 200 images belonging to 8 classes.
Found 200 images belonging to 1 classes.


Sometimes it's helpful to have just the filenames, without the path.

In [8]:
"""
# [Linux Environment]
raw_filenames = [f.split('/')[-1] for f in filenames]
raw_val_filenames = [f.split('/')[-1] for f in val_filenames]
raw_test_filenames = [f.split('/')[-1] for f in test_filenames]
"""
# [Windows Environment]
raw_filenames = [f.split('\\')[-1] for f in filenames]
raw_val_filenames = [f.split('\\')[-1] for f in val_filenames]
raw_test_filenames = [f.split('\\')[-1] for f in test_filenames]

In [9]:
len(raw_filenames), len(raw_val_filenames), len(raw_test_filenames)

(400, 200, 200)

In [10]:
input_shape = next(batches)[0].shape[1:]
print(input_shape)

(3, 224, 224)


In [12]:
trn_data = get_data(train_path)
val_data = get_data(valid_path)
test_data = get_data(test_path)

Found 400 images belonging to 8 classes.
Found 200 images belonging to 8 classes.
Found 200 images belonging to 1 classes.


In [13]:
save_array(result_path+'trn_data', trn_data)
save_array(result_path+'val_data', val_data)
save_array(result_path+'test_data', test_data)

In [14]:
trn_data = load_array(result_path+'trn_data')
val_data = load_array(result_path+'val_data')
test_data = load_array(result_path+'test_data')

## Basic VGG

We start with our usual VGG approach.  We will be using VGG with batch normalization.  We explained how to add batch normalization to VGG in the [imagenet_batchnorm notebook](https://github.com/fastai/courses/blob/master/deeplearning1/nbs/imagenet_batchnorm.ipynb).  VGG with batch normalization is implemented in [vgg_bn.py](https://github.com/fastai/courses/blob/master/deeplearning1/nbs/vgg16bn.py), and there is a version of ``vgg_ft`` (our fine tuning function) with batch norm called ``vgg_ft_bn`` in [utils.py](https://github.com/fastai/courses/blob/master/deeplearning1/nbs/utils.py).

### Initial model

First we create a simple fine-tuned VGG model to be our starting point.

In [15]:
from vgg16bn_ted import Vgg16BN
model = vgg_ft_bn(8)

In [16]:
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
model.fit(trn_data, trn_labels, epochs=2, verbose=2, batch_size=batch_size,
         validation_data=(val_data, val_labels))

Train on 400 samples, validate on 200 samples
Epoch 1/2


KeyboardInterrupt: 

In [ ]:
model.save_weights(model_path+'ft1.h5')

### Precompute convolutional output

We pre-compute the output of the last convolution layer of VGG, since we're unlikely to need to fine-tune those layers. (All following analysis will be done on just the pre-computed convolutional features.)

In [ ]:
model.load_weights(model_path+'ft1.h5')

In [ ]:
conv_layers, fc_layers = split_at(model, Conv2D)

In [ ]:
conv_model = Sequential(conv_layers)

In [20]:
conv_features = conv_model.predict(trn_data, batch_size=batch_size)
conv_val_features = conv_model.predict(val_data, batch_size=batch_size)
conv_test_features = conv_model.predict(test_data, batch_size=batch_size)

In [ ]:
save_array(result_path+'conv_features.dat', conv_features)
save_array(result_path+'conv_val_features.dat', conv_val_features)
save_array(result_path+'conv_test_features.dat', conv_test_features)

In [ ]:
conv_features = load_array(result_path+'conv_features.dat')
conv_val_features = load_array(result_path+'conv_val_features.dat')
conv_test_features = load_array(result_path+'conv_test_features.dat')

In [ ]:
conv_val_features.shape

### Train model

We can now create our first baseline model - a simple 3-layer FC net.

In [ ]:
def get_bn_model():
    model = Sequential([
        MaxPooling2D(input_shape=conv_model.layers[-1].output_shape[1:]),
        BatchNormalization(axis=1),
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dense(8, activation='softmax')
    ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
bn_model = get_bn_model()

In [ ]:
bn_model.fit(conv_features, trn_labels, epochs=4, verbose=2,
            validation_data=(conv_val_features, val_labels))

In [ ]:
bn_model.lr = 1e-4

In [ ]:
bn_model.fit(conv_features, trn_labels, epochs=6, verbose=2,
            validation_data=(conv_val_features, val_labels))

In [ ]:
bn_model.save_weights(model_path+'conv_512_1.h5')

In [ ]:
bn_model.evaluate(conv_val_features, val_labels)

In [ ]:
bn_model.load_weights(model_path+'conv_512_1.h5')

## Multi-input

The images are of different sizes, which are likely to represent the boat they came from (since different boats will use different cameras). Perhaps this creates some data leakage that we can take advantage of to get a better Kaggle leaderboard position? To find out, first we create arrays of the file sizes for each image:

In [ ]:
## Multi-input

The images are of different sizes, which are likely to represent the boat they came from (since different boats will use different cameras). Perhaps this creates some data leakage that we can take advantage of to get a better Kaggle leaderboard position? To find out, first we create arrays of the file sizes for each image: